#Homework 2 - Spark

## Lettura dei dati

In [0]:
import requests

from pyspark.sql.functions import isnan, when, count, isnull

In [0]:
# local_path_popolazione_regioni = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/popolazione_regioni.csv"
# local_path_dati_covid_regioni = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/dati_covid_regioni.csv"
# local_path_dati_vaccini_somministrazioni = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/dati_vaccini_somministrazioni.csv"
# local_path_dati_vaccini_consegne = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/dati_vaccini_consegne.csv"

local_path_popolazione_regioni = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/popolazione_regioni.csv"
local_path_dati_covid_regioni = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/dati_covid_regioni.csv"
local_path_dati_vaccini_somministrazioni = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/dati_vaccini_somministrazioni.csv"
local_path_dati_vaccini_consegne = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/dati_vaccini_consegne.csv"

popolazione_regioni_URL = "https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-statistici-riferimento/popolazione-istat-regione-range.csv"
dati_covid_regioni_URL = "https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv"
dati_vaccini_somministrazioni_URL = "https://raw.githubusercontent.com/italia/covid19-opendata-vaccini/master/dati/somministrazioni-vaccini-latest.csv"
dati_vaccini_consegne_URL = "https://raw.githubusercontent.com/italia/covid19-opendata-vaccini/master/dati/consegne-vaccini-latest.csv"

dbutils.fs.put(local_path_popolazione_regioni, requests.get(popolazione_regioni_URL).text, True)
dbutils.fs.put(local_path_dati_covid_regioni, requests.get(dati_covid_regioni_URL).text, True)
dbutils.fs.put(local_path_dati_vaccini_somministrazioni, requests.get(dati_vaccini_somministrazioni_URL).text, True)
dbutils.fs.put(local_path_dati_vaccini_consegne, requests.get(dati_vaccini_consegne_URL).text, True)

popolazione_regioni = spark.read.option("inferSchema", "true") \
                                .option("header", "true") \
                                .csv(local_path_popolazione_regioni)
dati_covid_regioni = spark.read.option("inferSchema", "true") \
                                .option("header", "true") \
                                .csv(local_path_dati_covid_regioni)
dati_vaccini_somministrazioni = spark.read.option("inferSchema", "true") \
                                          .option("header", "true") \
                                          .csv(local_path_dati_vaccini_somministrazioni)
dati_vaccini_consegne = spark.read.option("inferSchema", "true") \
                                  .option("header", "true") \
                                  .csv(local_path_dati_vaccini_consegne)

Wrote 17300 bytes.
Wrote 1481207 bytes.
Wrote 3759833 bytes.
Wrote 54925 bytes.

## Preprocessing

### Popolazione Regioni
Si eliminano le colonne che non sono di interesse.

In [0]:
popolazione_regioni = popolazione_regioni.drop('codice_regione', 'latitudine_regione', 'longitudine_regione')
popolazione_regioni.show()

# popolazione_regioni.groupBy('denominazione_regione').sum('totale_generale').orderBy('sum(totale_generale)', ascending=False).show()

+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
codice_nuts_1|descrizione_nuts_1|codice_nuts_2|denominazione_regione|sigla_regione|range_eta|totale_genere_maschile|totale_genere_femminile|totale_generale|
+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
 ITF| Sud| ITF1| Abruzzo| ABR| 0-15| 87423| 82474| 169897|
 ITF| Sud| ITF1| Abruzzo| ABR| 16-19| 24402| 22349| 46751|
 ITF| Sud| ITF1| Abruzzo| ABR| 20-29| 67472| 62611| 130083|
 ITF| Sud| ITF1| Abruzzo| ABR| 30-39| 76472| 74241| 150713|
 ITF| Sud| ITF1| Abruzzo| ABR| 40-49| 94745| 95315| 190060|
 ITF| Sud| ITF1| Abruzzo| ABR| 50-59| 100048| 104109| 204157|
 ITF| Sud| ITF1| Abruzzo| ABR| 60-69| 80545| 87160| 167705|
 ITF| Sud| ITF1| Abruzzo| ABR| 70-79| 60810| 69762| 130572|
 ITF| Sud| ITF1| Abruzzo| ABR| 80-89| 34181| 50307| 84488|
 ITF| Sud| ITF1| Abruzzo| ABR| 90+| 5645| 13870| 19515|
 ITF| Sud| ITF5| Basilicata| BAS| 0-15| 36113| 33881| 69994|
 ITF| Sud| ITF5| Basilicata| BAS| 16-19| 11598| 10434| 22032|
 ITF| Sud| ITF5| Basilicata| BAS| 20-29| 32068| 29115| 61183|
 ITF| Sud| ITF5| Basilicata| BAS| 30-39| 33556| 31648| 65204|
 ITF| Sud| ITF5| Basilicata| BAS| 40-49| 39260| 39207| 78467|
 ITF| Sud| ITF5| Basilicata| BAS| 50-59| 43150| 44672| 87822|
 ITF| Sud| ITF5| Basilicata| BAS| 60-69| 35360| 37457| 72817|
 ITF| Sud| ITF5| Basilicata| BAS| 70-79| 24050| 27755| 51805|
 ITF| Sud| ITF5| Basilicata| BAS| 80-89| 14507| 21600| 36107|
 ITF| Sud| ITF5| Basilicata| BAS| 90+| 2488| 5335| 7823|
+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
only showing top 20 rows

Si verifica che il formato dei dati sia corretto.

In [0]:
popolazione_regioni.dtypes

Out[37]: [('codice_nuts_1', 'string'),
 ('descrizione_nuts_1', 'string'),
 ('codice_nuts_2', 'string'),
 ('denominazione_regione', 'string'),
 ('sigla_regione', 'string'),
 ('range_eta', 'string'),
 ('totale_genere_maschile', 'int'),
 ('totale_genere_femminile', 'int'),
 ('totale_generale', 'int')]

Si verifica la correttezza delle colonne categoriche.

In [0]:
num_regioni = popolazione_regioni.groupBy('denominazione_regione') \
                .count() \
                .count()
print('Il numero di regioni è:', num_regioni)
print()
popolazione_regioni.groupBy('codice_nuts_1', 'descrizione_nuts_1', 'codice_nuts_2', 'denominazione_regione', 'sigla_regione') \
                    .count() \
                    .drop('count') \
                    .orderBy('descrizione_nuts_1') \
                    .show(num_regioni)

Il numero di regioni è: 21

+-------------+------------------+-------------+---------------------+-------------+
codice_nuts_1|descrizione_nuts_1|codice_nuts_2|denominazione_regione|sigla_regione|
+-------------+------------------+-------------+---------------------+-------------+
 ITI| Centro| ITI4| Lazio| LAZ|
 ITI| Centro| ITI3| Marche| MAR|
 ITI| Centro| ITI2| Umbria| UMB|
 ITI| Centro| ITI1| Toscana| TOS|
 ITG| Isole| ITG1| Sicilia| SIC|
 ITG| Isole| ITG2| Sardegna| SAR|
 ITH| Nord-Est| ITH5| Emilia-Romagna| EMR|
 ITH| Nord-Est| ITH1| Bolzano| BOL|
 ITH| Nord-Est| ITH4| Friuli Venezia Gi...| FVG|
 ITH| Nord-Est| ITH2| Trento| TRE|
 ITH| Nord-Est| ITH3| Veneto| VEN|
 ITC| Nord-Ovest| ITC1| Piemonte| PIE|
 ITC| Nord-Ovest| ITC2| Valle d'Aosta| VDA|
 ITC| Nord-Ovest| ITC3| Liguria| LIG|
 ITC| Nord-Ovest| ITC4| Lombardia| LOM|
 ITF| Sud| ITF1| Abruzzo| ABR|
 ITF| Sud| ITF2| Molise| MOL|
 ITF| Sud| ITF3| Campania| CAM|
 ITF| Sud| ITF5| Basilicata| BAS|
 ITF| Sud| ITF4| Puglia| PUG|
 ITF| Sud| ITF6| Calabria| CAL|
+-------------+------------------+-------------+---------------------+-------------+

In [0]:
popolazione_regioni.groupBy('range_eta') \
                    .count() \
                    .orderBy('range_eta') \
                    .show(num_regioni)

+---------+-----+
range_eta|count|
+---------+-----+
 0-15| 21|
 16-19| 21|
 20-29| 21|
 30-39| 21|
 40-49| 21|
 50-59| 21|
 60-69| 21|
 70-79| 21|
 80-89| 21|
 90+| 21|
+---------+-----+

Si verifica la correttezza delle colonne numeriche.

In [0]:
popolazione_regioni.describe(['totale_genere_maschile', 'totale_genere_femminile', 'totale_generale']) \
                    .show()

+-------+----------------------+-----------------------+-----------------+
summary|totale_genere_maschile|totale_genere_femminile| totale_generale|
+-------+----------------------+-----------------------+-----------------+
 count| 210| 210| 210|
 mean| 138333.79047619048| 145673.29523809525|284007.0857142857|
 stddev| 151728.9786894938| 152578.20019731886|303588.4857882117|
 min| 457| 1307| 1764|
 max| 795710| 796399| 1592109|
+-------+----------------------+-----------------------+-----------------+

Si verifica la presenza di *missing values*.

In [0]:
popolazione_regioni.select([count(when(isnull(c), c)).alias(c) for c in popolazione_regioni.columns]).show()

+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
codice_nuts_1|descrizione_nuts_1|codice_nuts_2|denominazione_regione|sigla_regione|range_eta|totale_genere_maschile|totale_genere_femminile|totale_generale|
+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+
 0| 0| 0| 0| 0| 0| 0| 0| 0|
+-------------+------------------+-------------+---------------------+-------------+---------+----------------------+-----------------------+---------------+

### Dati Covid Regioni
Si eliminano le colonne che non sono di interesse.

In [0]:
dati_covid_regioni = dati_covid_regioni.drop('stato', 'codice_regione', 'lat', 'long', 'note_test', 'note_casi', 'note',
                                             'casi_da_sospetto_diagnostico', 'casi_da_screening',
                                             'casi_testati', 'ingressi_terapia_intensiva', 
                                             'totale_positivi_test_molecolare', 'totale_positivi_test_antigenico_rapido',
                                             'tamponi_test_molecolare', 'tamponi_test_antigenico_rapido')
dati_covid_regioni.show()

+-------------------+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
 data|denominazione_regione|ricoverati_con_sintomi|terapia_intensiva|totale_ospedalizzati|isolamento_domiciliare|totale_positivi|variazione_totale_positivi|nuovi_positivi|dimessi_guariti|deceduti|totale_casi|tamponi|codice_nuts_1|codice_nuts_2|
+-------------------+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
2020-02-24 18:00:00| Abruzzo| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 5| null| null|
2020-02-24 18:00:00| Basilicata| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| null| null|
2020-02-24 18:00:00| Calabria| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| null| null|
2020-02-24 18:00:00| Campania| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 10| null| null|
2020-02-24 18:00:00| Emilia-Romagna| 10| 2| 12| 6| 18| 0| 18| 0| 0| 18| 148| null| null|
2020-02-24 18:00:00| Friuli Venezia Gi...| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 58| null| null|
2020-02-24 18:00:00| Lazio| 1| 1| 2| 0| 2| 0| 2| 1| 0| 3| 124| null| null|
2020-02-24 18:00:00| Liguria| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| null| null|
2020-02-24 18:00:00| Lombardia| 76| 19| 95| 71| 166| 0| 166| 0| 6| 172| 1463| null| null|
2020-02-24 18:00:00| Marche| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 16| null| null|
2020-02-24 18:00:00| Molise| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| null| null|
2020-02-24 18:00:00| P.A. Bolzano| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| null| null|
2020-02-24 18:00:00| P.A. Trento| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 3| null| null|
2020-02-24 18:00:00| Piemonte| 2| 0| 2| 1| 3| 0| 3| 0| 0| 3| 141| null| null|
2020-02-24 18:00:00| Puglia| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| null| null|
2020-02-24 18:00:00| Sardegna| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| null| null|
2020-02-24 18:00:00| Sicilia| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 5| null| null|
2020-02-24 18:00:00| Toscana| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 140| null| null|
2020-02-24 18:00:00| Umbria| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| null| null|
2020-02-24 18:00:00| Valle d'Aosta| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 7| null| null|
+-------------------+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
only showing top 20 rows

Si verifica che il formato dei dati sia corretto.

In [0]:
dati_covid_regioni.dtypes

Out[47]: [('data', 'timestamp'),
 ('denominazione_regione', 'string'),
 ('ricoverati_con_sintomi', 'int'),
 ('terapia_intensiva', 'int'),
 ('totale_ospedalizzati', 'int'),
 ('isolamento_domiciliare', 'int'),
 ('totale_positivi', 'int'),
 ('variazione_totale_positivi', 'int'),
 ('nuovi_positivi', 'int'),
 ('dimessi_guariti', 'int'),
 ('deceduti', 'int'),
 ('totale_casi', 'int'),
 ('tamponi', 'int'),
 ('codice_nuts_1', 'string'),
 ('codice_nuts_2', 'string')]

Si verifica la correttezza delle colonne categoriche.

In [0]:
num_regioni = dati_covid_regioni.groupBy('denominazione_regione') \
                .count() \
                .count()
print('Il numero di regioni è:', num_regioni)
print()
dati_covid_regioni.groupBy('codice_nuts_1', 'codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .orderBy('codice_nuts_1') \
                    .show(50)

Il numero di regioni è: 21

+-------------+-------------+---------------------+
codice_nuts_1|codice_nuts_2|denominazione_regione|
+-------------+-------------+---------------------+
 null| null| Lazio|
 null| null| Sicilia|
 null| null| Emilia-Romagna|
 null| null| Toscana|
 null| null| Puglia|
 null| null| Abruzzo|
 null| null| Sardegna|
 null| null| Umbria|
 null| null| Piemonte|
 null| null| P.A. Bolzano|
 null| null| Molise|
 null| null| Calabria|
 null| null| Friuli Venezia Gi...|
 null| null| Valle d'Aosta|
 null| null| P.A. Trento|
 null| null| Marche|
 null| null| Basilicata|
 null| null| Veneto|
 null| null| Campania|
 null| null| Lombardia|
 null| null| Liguria|
 1| null| P.A. Bolzano|
 172410| ITI| Umbria|
 500340| ITH| P.A. Bolzano|
 ITC| ITC2| Valle d'Aosta|
 ITC| ITC3| Liguria|
 ITC| ITC1| Piemonte|
 ITC| ITC4| Lombardia|
 ITF| ITF6| Calabria|
 ITF| ITF2| Molise|
 ITF| ITF5| Basilicata|
 ITF| ITF1| Abruzzo|
 ITF| ITF3| Campania|
 ITF| ITF4| Puglia|
 ITG| ITG1| Sicilia|
 ITG| ITG2| Sardegna|
 ITH| ITH3| Veneto|
 ITH| ITH5| Emilia-Romagna|
 ITH| ITH2| P.A. Trento|
 ITH| ITH1| P.A. Bolzano|
 ITH| ITH4| Friuli Venezia Gi...|
 ITI| ITI1| Toscana|
 ITI| ITI2| Umbria|
 ITI| ITI3| Marche|
 ITI| ITI4| Lazio|
+-------------+-------------+---------------------+

Si possono fare alcune osservazioni:
- per alcune entry mancano i codici nuts;
- per aclune entry i codici nuts sono sbagliati (problema dovuto alla lettura del file csv);
- alcune regioni (Bolzano e Trento) hanno una denominazione diversa rispetto alla tabella `popolazione_regioni`;

In [0]:
join1 = popolazione_regioni.join(dati_covid_regioni, on='denominazione_regione', how='anti') \
                    .groupBy('codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .withColumnRenamed('denominazione_regione', 'popolazione_regioni_denominazione_regione')

join2 = dati_covid_regioni.join(popolazione_regioni, on='denominazione_regione', how='anti') \
                    .groupBy('codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .withColumnRenamed('denominazione_regione', 'dati_covid_regioni_denominazione_regione')

join1.join(join2, on='codice_nuts_2').show()

dati_covid_regioni = dati_covid_regioni.replace('P.A. Trento', value='Trento', subset='denominazione_regione')
dati_covid_regioni = dati_covid_regioni.replace('P.A. Bolzano', value='Bolzano', subset='denominazione_regione')

+-------------+-----------------------------------------+----------------------------------------+
codice_nuts_2|popolazione_regioni_denominazione_regione|dati_covid_regioni_denominazione_regione|
+-------------+-----------------------------------------+----------------------------------------+
 ITH1| Bolzano| P.A. Bolzano|
 ITH2| Trento| P.A. Trento|
+-------------+-----------------------------------------+----------------------------------------+

In [0]:
dati_covid_regioni = dati_covid_regioni.drop('codice_nuts_1', 'codice_nuts_2')

data = popolazione_regioni.groupBy('codice_nuts_1', 'codice_nuts_2', 'denominazione_regione') \
                          .count() \
                          .drop('count')

dati_covid_regioni = dati_covid_regioni.join(data, on=['denominazione_regione'], how='left')

dati_covid_regioni.groupBy('codice_nuts_1', 'codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .orderBy('codice_nuts_1').show(num_regioni)

+-------------+-------------+---------------------+
codice_nuts_1|codice_nuts_2|denominazione_regione|
+-------------+-------------+---------------------+
 ITC| ITC2| Valle d'Aosta|
 ITC| ITC3| Liguria|
 ITC| ITC1| Piemonte|
 ITC| ITC4| Lombardia|
 ITF| ITF4| Puglia|
 ITF| ITF1| Abruzzo|
 ITF| ITF6| Calabria|
 ITF| ITF2| Molise|
 ITF| ITF5| Basilicata|
 ITF| ITF3| Campania|
 ITG| ITG2| Sardegna|
 ITG| ITG1| Sicilia|
 ITH| ITH3| Veneto|
 ITH| ITH5| Emilia-Romagna|
 ITH| ITH1| Bolzano|
 ITH| ITH2| Trento|
 ITH| ITH4| Friuli Venezia Gi...|
 ITI| ITI1| Toscana|
 ITI| ITI2| Umbria|
 ITI| ITI3| Marche|
 ITI| ITI4| Lazio|
+-------------+-------------+---------------------+

Si verifica la presenza di *missing values*.

In [0]:
dati_covid_regioni.select([count(when(isnull(c), c)).alias(c) for c in dati_covid_regioni.drop('data').columns]).show()

+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
denominazione_regione|ricoverati_con_sintomi|terapia_intensiva|totale_ospedalizzati|isolamento_domiciliare|totale_positivi|variazione_totale_positivi|nuovi_positivi|dimessi_guariti|deceduti|totale_casi|tamponi|codice_nuts_1|codice_nuts_2|
+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+---------------------+----------------------+-----------------+--------------------+----------------------+---------------+--------------------------+--------------+---------------+--------+-----------+-------+-------------+-------------+

### Dati Somministrazioni Vaccini